In [78]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import random
import tensorflow.keras.utils as utils
import pydot
from tensorflow.keras import regularizers

In [79]:
dataset_dir = "Datasets/first-order-theorem-proving_csv.csv"

In [80]:
df = pd.read_csv(dataset_dir, index_col=None)
df = df.fillna(df.mean())
df.head(100)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V43,V44,V45,V46,V47,V48,V49,V50,V51,Class
0,1.48950,0.81688,2.1168,1.7836,1.53460,-0.30543,-1.24770,-0.14537,-0.59356,-0.23316,...,0.026975,1.83230,0.97743,2.6676,1.68030,-0.48278,1.46300,0.27015,-1.55590,6
1,1.48950,0.81688,2.1168,1.7895,1.54030,-0.31850,-1.24770,-0.14537,-0.59356,-0.23316,...,0.026975,1.83230,0.97743,2.6676,1.69740,-0.48278,1.48010,0.21361,-1.55590,1
2,1.48950,0.81688,2.1168,1.7836,1.53460,-0.30543,-1.24770,-0.14537,-0.59356,-0.23316,...,0.026975,1.83230,0.97743,2.6676,1.69170,-0.48278,1.47440,0.23245,-1.55590,6
3,1.48950,0.81688,2.1168,1.7836,1.53460,-0.30543,-1.24770,-0.14537,-0.59356,-0.23316,...,0.026975,1.83230,0.97743,2.6676,1.65750,-0.48278,1.44010,0.34553,-1.55590,6
4,1.48950,0.81688,2.1168,1.7836,1.53460,-0.30543,-1.24770,-0.14537,-0.59356,-0.23316,...,0.026975,1.83230,0.97743,2.6676,1.66320,-0.48278,1.44580,0.32669,-1.55590,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.97422,-2.82510,2.7381,-1.0652,-1.09820,-0.32988,1.10730,-0.28157,0.76331,-0.50533,...,-0.112290,-0.56363,-1.79950,3.4576,-0.55678,-0.48990,-0.66229,-0.16876,0.71854,3
96,-0.72640,-2.80740,2.7381,-1.0652,-0.81894,0.58234,0.50195,-0.14537,0.62074,-0.50533,...,-0.112290,-0.57834,-1.88710,3.4576,-0.55678,-0.48990,-0.47584,0.10746,0.44636,5
97,-0.98776,-2.92290,2.7381,-1.0652,-1.11050,-0.38680,1.14050,-0.14537,0.79415,-0.50533,...,-0.112290,-0.56520,-1.86910,3.4576,-0.55678,-0.48990,-0.67537,-0.20043,0.74140,5
98,-0.98421,-3.41720,2.7381,-1.0652,-1.10650,-0.41329,1.14730,-0.28157,0.77468,-0.64141,...,-0.112290,-0.56491,-2.20540,3.4576,-0.55678,-0.48990,-0.67292,-0.19449,0.73711,3


In [81]:
label_column = df.pop('Class')
label_column = np.asarray(label_column)
label_column = label_column - 1
df.insert(0, 'label', label_column)
df

,label,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51
0,5,1.48950,0.81688,2.11680,1.78360,1.53460,-0.305430,-1.24770,-0.14537,-0.593560,...,-0.16374,0.026975,1.83230,0.97743,2.66760,1.68030,-0.48278,1.46300,0.27015,-1.55590
1,0,1.48950,0.81688,2.11680,1.78950,1.54030,-0.318500,-1.24770,-0.14537,-0.593560,...,-0.16374,0.026975,1.83230,0.97743,2.66760,1.69740,-0.48278,1.48010,0.21361,-1.55590
2,5,1.48950,0.81688,2.11680,1.78360,1.53460,-0.305430,-1.24770,-0.14537,-0.593560,...,-0.16374,0.026975,1.83230,0.97743,2.66760,1.69170,-0.48278,1.47440,0.23245,-1.55590
3,5,1.48950,0.81688,2.11680,1.78360,1.53460,-0.305430,-1.24770,-0.14537,-0.593560,...,-0.16374,0.026975,1.83230,0.97743,2.66760,1.65750,-0.48278,1.44010,0.34553,-1.55590
4,5,1.48950,0.81688,2.11680,1.78360,1.53460,-0.305430,-1.24770,-0.14537,-0.593560,...,-0.16374,0.026975,1.83230,0.97743,2.66760,1.66320,-0.48278,1.44580,0.32669,-1.55590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6113,5,1.38650,0.83152,-0.23966,1.16080,0.92316,0.763810,-1.12100,-0.55397,-0.539690,...,-0.16374,-0.112290,-0.36328,0.99018,-0.54541,-0.33836,-0.21738,-0.56571,-0.46486,0.71176
6114,5,-0.50624,-0.58481,-0.33987,-0.49442,-0.40892,-0.090004,0.39949,0.67183,0.059521,...,1.88690,0.305510,-0.57834,-1.34880,-0.54541,-0.55678,-0.48990,-0.53790,-0.46486,0.68376
6115,4,-0.59253,0.73765,-0.62354,-0.56530,-0.73434,-0.599120,0.88843,-0.41777,-0.334510,...,-0.16374,-0.112290,-0.28034,0.91559,-0.43858,-0.33992,-0.48990,-0.54816,-0.13433,0.59309
6116,5,-0.51839,0.43125,-0.49860,-0.39309,-0.48333,-0.642260,0.68166,-0.28157,-0.204090,...,-0.16374,0.026975,-0.43072,0.34074,-0.19947,-0.40686,-0.34960,-0.37141,-0.27919,0.45935


In [82]:
df.isnull().values.any()

False

In [83]:
df = df.sample(frac=1, random_state=41)

In [84]:
col_num = len(df.columns)
row_num = len(df.index)
print("Number of columns is: {} and number of rows is: {}".format(col_num, row_num))

Number of columns is: 52 and number of rows is: 6118


In [85]:
trainx = df.iloc[:int(0.8*row_num), 1:(col_num)]
trainy = df.iloc[:int(0.8*row_num), 0]

testx = df.iloc[int(0.8*row_num):, 1:(col_num)]
testy = df.iloc[int(0.8*row_num):, 0]

In [86]:
trainy

3010    5
4888    2
3508    0
2485    4
2375    5
       ..
5851    4
2330    2
1902    5
1289    0
1662    1
Name: label, Length: 4894, dtype: int64

In [87]:
scaler = StandardScaler()
trainx_scaled = pd.DataFrame(scaler.fit_transform(trainx), columns = trainx.columns, index = trainx.index)
textx_scaled = pd.DataFrame(scaler.transform(testx), columns = testx.columns, index = testx.index)

In [93]:
trainx_scaled

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51
3010,-1.049193,-2.646142,-0.978325,-0.992785,-1.188927,-0.243668,1.158136,-0.009270,1.403881,-0.642189,...,-0.175921,-0.127991,-0.578185,-1.943720,-0.532895,-0.556443,-0.488727,-0.649132,-0.456475,0.794834
4888,-0.033691,0.012195,-0.180647,0.122679,-0.095518,-0.687557,0.357672,-0.155612,-0.114726,-0.373120,...,-0.175921,-0.127991,-0.578185,0.932467,0.030919,-0.556443,-0.488727,-0.843227,-0.456475,0.989027
3508,-0.827254,-3.278007,2.341687,-0.729316,2.276311,-0.660671,-1.770774,0.868795,-0.200183,-0.642189,...,-0.175921,-0.127991,-0.423461,-2.113639,1.753357,-0.399262,-0.291596,1.744763,-0.456475,-1.600159
2485,-0.801216,-0.977780,-0.372338,-0.733599,-0.762407,-0.381209,0.831765,0.429763,0.387124,-0.238591,...,-0.175921,-0.040767,-0.517134,-0.504527,0.224826,-0.515104,-0.436871,-0.800978,0.007996,0.798799
2375,-0.793144,-0.882079,-0.517617,-0.726277,-0.909346,-0.307321,0.933698,0.429763,0.421518,-0.373120,...,-0.175921,0.046463,-0.525240,-0.004216,-0.356880,-0.502662,-0.421276,-0.788260,0.234066,0.714065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5851,-0.136939,-0.003997,-0.575371,0.031681,-0.184713,-0.283433,0.277217,0.429763,0.232387,-0.373120,...,-0.175921,-0.040767,-0.532515,-0.457692,-0.441797,-0.533251,-0.459640,-0.155724,0.362558,0.040302
2330,-0.846590,-0.909454,-0.598472,-0.829477,-1.028852,-0.246942,1.016491,0.429763,0.445400,-0.373120,...,-0.175921,-0.040767,-0.549426,0.070240,-0.418145,-0.527225,-0.452084,-0.813370,0.200036,0.750023
1902,-0.474686,0.404901,-0.435462,-0.350177,-0.443151,-0.625379,0.643445,-0.301956,-0.238084,2.317601,...,-0.175921,0.046463,-0.508078,0.062118,-0.222117,-0.485225,-0.399403,-0.552079,-0.304070,0.649195
1289,-0.436391,-0.652378,-0.023843,-0.265311,-0.380762,-0.029373,0.351604,-0.301956,-0.010245,-0.507659,...,0.185452,0.046463,-0.578185,-1.883398,-0.045789,-0.556443,-0.488727,0.107476,-0.456475,0.037884


In [94]:
import random
def create_rand_list(max_val, count):
    randomlist = random.sample(range(0, max_val + 1), count)
    return randomlist

In [95]:
def get_MICS_model(inp_size, drop_out, hidden_num = 6, hidden_size=64):
    inputs = keras.layers.Input(shape=(inp_size), name="input")
        
    h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(inputs)
    h = keras.layers.Dropout(drop_out)(h)
    for hidden in range(hidden_num):
        h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
        h = keras.layers.Dropout(drop_out)(h) 

    outputs = keras.layers.Dense(6, activation="sigmoid")(h)    
    return keras.Model(inputs=[inputs], outputs = outputs)

In [96]:
df["label"].value_counts()

5    2554
0    1089
2     748
4     624
3     617
1     486
Name: label, dtype: int64

In [97]:
inp_size = len(trainx.columns)
MICS_model = get_MICS_model(inp_size, drop_out = 0.25)
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25), 
        keras.callbacks.ReduceLROnPlateau("val_loss", factor = 0.5, patience=10,
                                         verbose = 2, mode = "auto", 
                                          min_lr = 1e-6)]
MICS_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
history = MICS_model.fit(x = [trainx_scaled], y = trainy.values,  
                         validation_data = ([textx_scaled], testy.values),
                         epochs=300, batch_size = 32, callbacks=callback)
training_val_accuracy = history.history["val_accuracy"]
best_row_index = np.argmax(training_val_accuracy)
best_val_accuracy = training_val_accuracy[best_row_index]
best_val_accuracy
#min_losses3[c].append(best_val_loss)
#i = i + 1

Epoch 1/300
153/153 [==============================] - 0s 2ms/step - loss: 1.8548 - accuracy: 0.4172 - val_loss: 1.6425 - val_accuracy: 0.4028
Epoch 2/300
153/153 [==============================] - 0s 1ms/step - loss: 1.6199 - accuracy: 0.4211 - val_loss: 1.5809 - val_accuracy: 0.4028
Epoch 3/300
153/153 [==============================] - 0s 2ms/step - loss: 1.5863 - accuracy: 0.4217 - val_loss: 1.5795 - val_accuracy: 0.4028
Epoch 4/300
153/153 [==============================] - 0s 1ms/step - loss: 1.5857 - accuracy: 0.4195 - val_loss: 1.5825 - val_accuracy: 0.4028
Epoch 5/300
153/153 [==============================] - 0s 1ms/step - loss: 1.5835 - accuracy: 0.4211 - val_loss: 1.5637 - val_accuracy: 0.4028
Epoch 6/300
153/153 [==============================] - 0s 2ms/step - loss: 1.5672 - accuracy: 0.4211 - val_loss: 1.5606 - val_accuracy: 0.4028
Epoch 7/300
153/153 [==============================] - 0s 1ms/step - loss: 1.5644 - accuracy: 0.4205 - val_loss: 1.5832 - val_accuracy: 0.4028

0.46078431606292725

In [99]:
k = MICS_model.predict(trainx_scaled)
k = np.argmax(k, axis=1)

In [104]:
k = pd.DataFrame(k)
k[0].value_counts()

0    2593
5    2301
Name: 0, dtype: int64